# Memória

Os LLM são essencialmente stateless, isto é não guardam histórico e cada interação é uma nova chamada na API. Cada transação é independente.
Chatbots aparentam ter memória porque o código que o implementa considera o histórico de toda conversa no contexto.

Langchain oferece diferentes tipos de gerenciadores de memória.
Abordaremos desses:
- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory

## Preparando o Ambiente

Para iniciar um trabalho sobre a API da OpenAI, vamos primeiro importar a biblioteca python da openai. Para facilitar o carregamento seguro da chave de API utilizada, também utilizaremos a biblioteca python-dotenv.

In [ ]:
%pip install openai
%pip install --upgrade langchain
%pip install langchain-community
%pip install tiktoken

In [ ]:
# Setup environment and import required libraries
import openai

In [ ]:
%set_env OPENAI_API_KEY=#Your API Key here

In [ ]:
llm_model="gpt-3.5-turbo"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

# Histórico de Navegação

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True # Esse comando permite ver o que a LLM está aplicando
)

In [ ]:
conversation.predict(input="Oi, me chamo Saulo")

In [ ]:
conversation.predict(input="Qual a resposta para a pergunta sobre o universo e tudo mais?")

In [ ]:
conversation.predict(input="qual meu nome?")

A variável memory armazena a memória da conversa até então. Vamos ver seu valor a seguir:

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

Se quisermos alterar um conteúdo, dizendo qual o contexto e memória pregresso da uma interação, pode ser definida uma variável nova para isso.

In [ ]:
memory = ConversationBufferMemory()

Com uma memória vazia, adicionar os elementos:

In [ ]:
memory.save_context(
    {"input": "Oi"},
    {"output": "Opa, novidades?"}
)

In [ ]:
print(memory.buffer)

In [ ]:
memory.save_context(
    {"input": "Nada demais"},
    {"output": "Massa"}
)

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

# Janela de Memória


In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=1)               

In [ ]:
memory.save_context(
    {"input": "Oi"},
    {"output": "Opa, novidades?"}
)

memory.save_context(
    {"input": "Nada demais"},
    {"output": "Massa"}
)

In [ ]:
memory.load_memory_variables({})

In [ ]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [ ]:
conversation.predict(input="Oi, meu nome é Saulo.")

In [ ]:
conversation.predict(input="Qual capital de Pernambuco?")

In [ ]:
conversation.predict(input="Qual meu nome?")

# Token

A memória também pode ser controlada usando a quantidade de tokens como referência.

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "IA é o que?!"},
                    {"output": "Arretada!"})
memory.save_context({"input": "Aprendizagem de máquina é o que?"},
                    {"output": "Demais!"})
memory.save_context({"input": "Chatbots são o que?"}, 
                    {"output": "Simpáticos!"})

In [ ]:
memory.load_memory_variables({})

# Resumo

Resumo para combinar de forma eficiente o conteúdo passado.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Oi"}, {"output": "Conta as novas"})
memory.save_context({"input": "Nada demais, só passando tempo"},
                    {"output": "Legal"})
memory.save_context({"input": "Qual a agenda de hoje?"}, 
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")

In [ ]:
memory.load_memory_variables({})